In [3]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

In [6]:
f = Path('../data/links_with_zipcodes_Text.csv')

In [7]:
ojd = pd.read_csv(f)
ojd

,media,pdf_link,Text,zipcode
0,7DIESACTUALITAT.COM,https://www.ojdinteractiva.es/muestra_acta/med...,http://www.ojdinteractiva.es Octubre - 2024\nA...,12500.0
1,7TELEVALENCIA.COM,https://www.ojdinteractiva.es/muestra_acta/med...,http://www.ojdinteractiva.es Octubre - 2024\nA...,46988.0
2,ACTUALIDADCASTELLON.COM,https://www.ojdinteractiva.es/muestra_acta/med...,http://www.ojdinteractiva.es Octubre - 2024\nA...,46007.0
3,ACTUALIDADVALENCIA.COM,https://www.ojdinteractiva.es/muestra_acta/med...,http://www.ojdinteractiva.es Octubre - 2024\nA...,46007.0
4,AGENDADEISA.COM,https://www.ojdinteractiva.es/muestra_acta/med...,http://www.ojdinteractiva.es Octubre - 2024\nA...,46112.0
...,...,...,...,...
333,URGELLTV.CAT,https://www.ojdinteractiva.es/muestra_acta/mit...,http://www.ojdinteractiva.es Octubre - 2024\nA...,25007.0
334,21RADIO.CAT,https://www.ojdinteractiva.es/muestra_acta/mit...,http://www.ojdinteractiva.es Octubre - 2024\nA...,3500.0
335,AUE.CAT,https://www.ojdinteractiva.es/muestra_acta/mit...,http://www.ojdinteractiva.es Octubre - 2024\nA...,25007.0
336,FEMBASE.CAT,https://www.ojdinteractiva.es/muestra_acta/mit...,http://www.ojdinteractiva.es Octubre - 2024\nA...,25001.0


In [59]:
def extract_visits_data(text):
    '''Extracts the data between "1. Cifras [some text] \n" and "2. [some text]"'''
    
    # Regular expression to find content between "1. Cifras [some text] \n" and "2. [some text]"
    pattern = r'VISITAS\s*PAGINAS\s*\n(.*?)2\. Secciones\s'
    
    # Use re.DOTALL to allow '.' to match newlines
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group(1).strip()  # Return the content between "1. Cifras ..." and "2. "
    else:
        pattern = r'VISITES\s*PÀGINES\s*\n(.*?)2\. Seccions\s'
        match = re.search(pattern, text, re.DOTALL)
        if match:
            return match.group(1).strip()
        else:
            return None

def form_data_from_text(text):
    t = extract_visits_data(text)
    xs = ['browser_uniques', 'visits', 'pages']
    ys = ['1_Month', 'daily_avg', 'mon_fri_avg', 'sat_sun_avg', 'pages_per_visit', 'avg_visit_duration', 'avg_interaction_time' ]
    data = {xs[0]: {}, xs[1]: {}, xs[2]: {}}
    
    if t is None:
        return None
    for id, text in enumerate(t.split('\n')):
        y_key = ys[id]
        if id < 4:
            nums = text.split(' ')[-3:]
            for x, num in enumerate(nums):
                x_key = xs[x]
                data[x_key][y_key] = num
        else:
            nums = text.split(' ')[-1:]
            for x, num in enumerate(nums):
                x_key = xs[x]
                data[x_key][y_key] = num
    return data
        


In [64]:
form_data_from_text(ojd.iloc[110].Text)

{'browser_uniques': {'1_Month': '4.774',
  'daily_avg': '164',
  'mon_fri_avg': '178',
  'sat_sun_avg': '125',
  'pages_per_visit': '1,59',
  'avg_visit_duration': '0:02:23',
  'avg_interaction_time': '0:01:04'},
 'visits': {'1_Month': '5.527',
  'daily_avg': '178',
  'mon_fri_avg': '194',
  'sat_sun_avg': '132'},
 'pages': {'1_Month': '8.806',
  'daily_avg': '284',
  'mon_fri_avg': '308',
  'sat_sun_avg': '215'}}

In [68]:
ojd['visits_data'] = ojd.Text.apply(form_data_from_text)

In [71]:
ojd[['media', 'zipcode', 'visits_data']].to_csv('../data/ojd_visits_data.csv', index=False)